In [ ]:
import time
import os
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

class GoogleMapScraper:
    def __init__(self):
        self.output_file_name = "test.csv"
        self.headless = False
        self.driver = None
        self.unique_check = []

    def config_driver(self):
        options = webdriver.ChromeOptions()
        if self.headless:
            options.add_argument("--headless")
            options.add_argument('--ignore-ssl-errors=yes')
            options.add_argument('--ignore-certificate-errors')
        
        # Set language preference to English
        options.add_argument("--lang=en-GB")
        
        # Set a specific port for remote debugging
        options.add_argument("--remote-debugging-port=9222")
        
        # Create a new instance of the Chrome WebDriver
        self.driver = webdriver.Chrome(options=options)


    def load_companies(self, url):
        print("Getting business info", url)
        self.driver.get(url)
        time.sleep(5)
        panel_xpath = "//div[contains(@class, 'm6QErb') and contains(@class, 'DxyBCb') and contains(@class, 'kA9KIf') and contains(@class, 'dS8AEf')]"
        try:
            scrollable_div = self.driver.find_element(By.XPATH, panel_xpath)
        except NoSuchElementException:
            print("Scrollable panel not found.")
            return

        flag = True
        i = 0
        while flag:
            print(f"Scrolling to page {i + 2}")
            self.driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + 6500', scrollable_div)
            time.sleep(2)

            if "You've reached the end of the list." in self.driver.page_source:
                flag = False

            self.get_business_info()
            i += 1

    def get_business_info(self):
        # Add a wait time before starting to scrape the business info
        WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'jJc9Ad')))
        
        try:
            for business in self.driver.find_elements(By.CLASS_NAME, 'jJc9Ad'):
                name = business.find_element(By.CLASS_NAME, 'd4r55').text
                try:
                    review = business.find_element(By.CLASS_NAME, 'wiI7pd').text
                except NoSuchElementException:
                    review = "No review found"
                # stars
                try:
                    stars_element = WebDriverWait(business, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, 'kvMYJc'))
                    )
                
                    rated_stars = stars_element.find_elements(By.CLASS_NAME, 'hCCjke.vzX5Ic.google-symbols.NhBTye')
                    non_rated_stars = stars_element.find_elements(By.CLASS_NAME, 'hCCjke.google-symbols.NhBTye')
                
                    rating = len(rated_stars)
                    print(f'Rating: {rating} stars')
                except NoSuchElementException:
                    print("Stars element not found for this business.")

                # Date check
                date_elements = business.find_elements(By.CLASS_NAME, 'rsqaWe')
                if date_elements:
                    date_text = date_elements[0].text
                    print(f'Date text: {date_text}')
                    date_parts = date_text.split(", ")
                    if len(date_parts) > 1:
                        date = date_parts[1]
                    else:
                        date = date_text
                    print(date)
                else:
                    print('Date element not found.')

                unique_id = "".join([name, str(rating), review, date])
                if unique_id not in self.unique_check:
                    data = [name, rating, review, date]
                    self.save_data(data)
                    self.unique_check.append(unique_id)
                    print(unique_id)

        except NoSuchElementException as e:
            print(f"An error occurred: {e}")

    def save_data(self, data):
        header = ['ID', 'Client_Name', 'Rating', 'Reviews', 'Date']
        file_exists = os.path.isfile(self.output_file_name)
        with open(self.output_file_name, 'a', newline='', encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            if not file_exists:
                writer.writerow(header)
            writer.writerow([len(self.unique_check)] + data)

url = "https://www.google.com/maps/place/Burj+Khalifa/@25.1972018,55.2718015,17z/data=!4m8!3m7!1s0x3e5f43348a67e24b:0xff45e502e1ceb7e2!8m2!3d25.197197!4d55.2743764!9m1!1b1!16zL20vMDNjbjB2?entry=ttu"

business_scraper = GoogleMapScraper()
business_scraper.config_driver()
business_scraper.load_companies(url)
